In [2]:
import pandas as pd

df = pd.read_csv('data/sabdab_nano_with_sequences.csv')
df.head()

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid,seq
0,8zer,A,NaN,0,C,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,True,IGHV1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...
1,8zer,D,NaN,0,I,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,True,IGHV1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...
2,8zer,F,NaN,0,B | E,protein | protein,NA | NA,spike protein s1 | spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,True,IGHV1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...
3,8zer,H,NaN,0,G,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,True,IGHV1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EVQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKERE...
4,8zer,J,NaN,0,I,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,True,IGHV1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...


In [3]:
TMP_ANARCI_DUMP_DIR = './data/anarci-out'

def build_anarci_outfile(id):
  return f'{TMP_ANARCI_DUMP_DIR}/{id}'

def build_anarci_csv_path(pdb, Hchain):
  return f'{TMP_ANARCI_DUMP_DIR}/{pdb}-{Hchain}_H.csv'

test_anarci_outfile = pd.read_csv(build_anarci_csv_path('8zer', 'A'))
test_anarci_outfile.head()

,Id,domain_no,hmm_species,chain_type,e-value,score,seqstart_index,seqend_index,identity_species,v_gene,...,104,105,106,107,108,109,110,111,112,113
0,Input sequence,0,human,H,9.400000e-45,143.0,0,119,NaN,NaN,...,G,Q,G,T,Q,V,T,V,S,S


In [11]:
CDR_H1_LOCATIONS  = ['26','27','28','29','30','31','32']
CDR_H2_LOCATIONS = [str(n) for n in range(52,57)]
CDR_H3_LOCATIONS = [str(n) for n in range(95, 103)]
test_anarci_outfile[CDR_H3_LOCATIONS].apply(lambda row: row.tolist(), axis=1).tolist()[0]

['W', 'A', 'C', 'S', 'S', 'G', 'G', 'D']

In [13]:
def extract_cdr_sequence(target_anarci_df, residue_mask):

    residues = target_anarci_df[residue_mask].apply(lambda row: row.tolist(), axis=1).tolist()[0]
    return "".join(residues)

extract_cdr_sequence(test_anarci_outfile, CDR_H3_LOCATIONS)

'WACSSGGD'

In [32]:
import os
import swifter

def transform_row_with_cdr_sequence(row, cdr_mask):
    pdb = row['pdb']
    Hchain = row['Hchain']

    anarci_outfilepath = build_anarci_csv_path(pdb, Hchain)
    if not os.path.exists(anarci_outfilepath):
        return None
    
    anarci_df = pd.read_csv(anarci_outfilepath)

    if not set(cdr_mask).issubset(anarci_df.columns):
        print('missing columns', set(cdr_mask).difference(anarci_df.columns), pdb, Hchain)
        return None
    

    return extract_cdr_sequence(anarci_df, cdr_mask)




In [30]:
df['CDR_H1'] = df[df['pdb'].notna() & df['Hchain'].notna()].swifter.apply(lambda row: transform_row_with_cdr_sequence(row, CDR_H1_LOCATIONS) ,axis=1)

Pandas Apply:   0%|          | 0/3309 [00:00<?, ?it/s]

In [25]:
df['CDR_H2'] = df[df['pdb'].notna() & df['Hchain'].notna()].swifter.apply(lambda row: transform_row_with_cdr_sequence(row, CDR_H2_LOCATIONS) ,axis=1)

Pandas Apply:   0%|          | 0/3309 [00:00<?, ?it/s]

In [33]:
df['CDR_H3'] = df[df['pdb'].notna() & df['Hchain'].notna()].swifter.apply(lambda row: transform_row_with_cdr_sequence(row, CDR_H3_LOCATIONS) ,axis=1)

Pandas Apply:   0%|          | 0/3309 [00:00<?, ?it/s]

missing columns {'96', '97', '98', '101', '99', '102', '100', '95'} 8c7j A
missing columns {'96', '97', '98', '101', '99', '102', '100', '95'} 6do1 F


In [34]:
df.head()

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,light_ctype,affinity,delta_g,affinity_method,temperature,pmid,seq,CDR_H1,CDR_H2,CDR_H3
0,8zer,A,NaN,0,C,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...,GYTYCSY,RRDGS,WACSSGGD
1,8zer,D,NaN,0,I,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...,GYTYCSY,RRDGS,WACSSGGD
2,8zer,F,NaN,0,B | E,protein | protein,NA | NA,spike protein s1 | spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...,GYTYCSY,RRDGS,WACSSGGD
3,8zer,H,NaN,0,G,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,NaN,EVQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKERE...,GYTYCSY,RRDGS,WACSSGGD
4,8zer,J,NaN,0,I,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...,GYTYCSY,RRDGS,WACSSGGD


In [36]:
df['CDR_concat'] = df['CDR_H1'] + df['CDR_H2'] + df['CDR_H3']

In [37]:
df.head()


,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,affinity,delta_g,affinity_method,temperature,pmid,seq,CDR_H1,CDR_H2,CDR_H3,CDR_concat
0,8zer,A,NaN,0,C,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...,GYTYCSY,RRDGS,WACSSGGD,GYTYCSYRRDGSWACSSGGD
1,8zer,D,NaN,0,I,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...,GYTYCSY,RRDGS,WACSSGGD,GYTYCSYRRDGSWACSSGGD
2,8zer,F,NaN,0,B | E,protein | protein,NA | NA,spike protein s1 | spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...,GYTYCSY,RRDGS,WACSSGGD,GYTYCSYRRDGSWACSSGGD
3,8zer,H,NaN,0,G,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,EVQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKERE...,GYTYCSY,RRDGS,WACSSGGD,GYTYCSYRRDGSWACSSGGD
4,8zer,J,NaN,0,I,protein,NaN,spike protein s1,VIRAL PROTEIN/IMMUNE SYSTEM,09/04/24,...,NaN,NaN,NaN,NaN,NaN,VQLVESGGGSVQAGGSLRLSCVASGYTYCSYDMSWYRQAPGKEREF...,GYTYCSY,RRDGS,WACSSGGD,GYTYCSYRRDGSWACSSGGD


In [39]:
df['CDR_concat'].unique, len(df)

(<bound method Series.unique of 0       GYTYCSYRRDGSWACSSGGD
 1       GYTYCSYRRDGSWACSSGGD
 2       GYTYCSYRRDGSWACSSGGD
 3       GYTYCSYRRDGSWACSSGGD
 4       GYTYCSYRRDGSWACSSGGD
                 ...         
 3316                     NaN
 3317    GSIFSLNNSGGRGTGWSPNY
 3318    GFPVNRYSAGDRNVGF--EY
 3319    GFDFDNYDSDGSWTLFHSEY
 3320    GFTFSNYSSGASCPAPFTAY
 Name: CDR_concat, Length: 3321, dtype: object>,
 3321)

In [41]:
df[['pdb', 'Hchain', 'CDR_concat']].head(20)

,pdb,Hchain,CDR_concat
0,8zer,A,GYTYCSYRRDGSWACSSGGD
1,8zer,D,GYTYCSYRRDGSWACSSGGD
2,8zer,F,GYTYCSYRRDGSWACSSGGD
3,8zer,H,GYTYCSYRRDGSWACSSGGD
4,8zer,J,GYTYCSYRRDGSWACSSGGD
5,8zes,E,GYTYCSYRRDGSWACSSGGD
6,8zes,B,GYTYCSYRRDGSWACSSGGD
7,8zes,F,GYTYCSYRRDGSWACSSGGD
8,8zes,H,GYTYCSYRRDGSWACSSGGD
9,8zes,J,GYTYCSYRRDGSWACSSGGD


In [42]:
df.to_csv('./data/sabdab_nano_with_cdr_sequences.csv', index=False)